In [1]:
import pandas as pd
import datetime
import csv

tmp_data_2017=pd.read_csv("E:\\5001\\KAGGLE\\2017_data.csv")
tmp_data_2018=pd.read_csv("E:\\5001\\KAGGLE\\2018_data.csv")
'''
This data comes from https://www.ncei.noaa.gov/access/search/data-search/global-hourly?bbox=22.363,113.843,22.212,114.067&startDate=2017-01-01T00:00:00&endDate=2018-12-31T23:59:59
Some weather information about Hong Kong Airport are inclued in this two files.
'''
#print(tmp_data_2017.head())

D:\Anaconda\Ana\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda\Ana\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (18,21,25,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'\nThis data comes from https://www.ncei.noaa.gov/access/search/data-search/global-hourly?bbox=22.363,113.843,22.212,114.067&startDate=2017-01-01T00:00:00&endDate=2018-12-31T23:59:59\nSome weather information about Hong Kong Airport are inclued in this two files.\n'

In [2]:
def dt_transformation(dt):
    dt["datetime_1"]=dt["DATE"].apply(lambda x:datetime.datetime.strptime(x[:13],"%Y-%m-%dT%H"))
    return dt
tmp_data_2017=dt_transformation(tmp_data_2017)
tmp_data_2018=dt_transformation(tmp_data_2018)
'''
Get corresponding Datetime information from each row of this two files.
'''
#print(tmp_data_2017.head())

'\nGet corresponding Datetime information from each row of this two files.\n'

In [3]:
from pytz import utc  
from pytz import timezone 

def timezone_transfermation(dt):
    cst_tz = timezone('Asia/Shanghai')  
    utc_tz = timezone('UTC') 
    dt["datetime"]=dt["datetime_1"].apply(lambda x:x.replace(tzinfo=utc_tz).astimezone(cst_tz))
    return dt
    
'''
Due to original TimeZone is UTC, we need to set the TimeZone as UTC+8 here.
'''    

def temperature_transfermation(dt):
    dt["Temp"]=dt["TMP"].apply(lambda x:float(x[2:5]))
    return dt
    
'''
Get Temperature of Hong Kong Airport.
'''
    
tmp_data_2017=timezone_transfermation(tmp_data_2017)
tmp_data_2018=timezone_transfermation(tmp_data_2018)
tmp_data_2017=temperature_transfermation(tmp_data_2017)
tmp_data_2018=temperature_transfermation(tmp_data_2018)
tmp_data_2017=tmp_data_2017[["datetime","Temp"]]
tmp_data_2018=tmp_data_2018[["datetime","Temp"]]

In [4]:
train_data=pd.read_csv("E:\\5001\\KAGGLE\\train.csv")
test_data=pd.read_csv("E:\\5001\\KAGGLE\\test.csv")

In [5]:
train_dataset=train_data.copy()
test_dataset=test_data.copy()
    
def datetime_transfermation(dataset):
    dataset['datetime']=dataset['date'].apply(lambda x:datetime.datetime.strptime(x,"%d/%m/%Y %H:00"))
    return dataset
    
def get_day_month_year(dataset):
    dataset['month']=dataset['datetime'].apply(lambda x:int(x.month))
    dataset['day']=dataset['datetime'].apply(lambda x:int(x.day))
    dataset['year']=dataset['datetime'].apply(lambda x:int(x.year))
    return dataset
    
def get_hour(dataset):
    dataset['hour']=dataset['datetime'].apply(lambda x:int(x.hour))
    return dataset

def get_hour_feature(dataset,feature_select=True):
    '''
    Every day is divided into different periods.
    '''
    if feature_select:
        def get_feature(x):
            time_range=[[0,1,2,3],[4,5,6,7],[8,9],[10,11,12],[13,14,15,16],[17,18,19],[20,21,22,23]]
            for i in range(len(time_range)):
                if x in time_range[i]:
                    return chr(i+97)
        dataset['hour_t']=dataset['hour'].apply(get_feature)
    else:
        dataset['hour_t']=dataset['hour'].apply(lambda x: chr(x+97))
    return dataset

def get_Weekday(dataset):
    dataset['Sun']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==6 else 0) 
    dataset['Sat']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==5 else 0)
    dataset['Fri']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==4 else 0)
    dataset['Thu']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==3 else 0) 
    dataset['Wed']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==2 else 0)
    dataset['Tue']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==1 else 0)
    dataset['Mon']=dataset['datetime'].apply(lambda x: 1 if x.weekday()==0 else 0)
    return dataset

def get_weekday_second_edition(dataset):
    dataset["weekday"]=dataset["datetime"].apply(lambda x: x.weekday())
    return dataset

def get_public_holiday(dataset):
    holiday_list= ['2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14',
           '2017-04-15','2017-04-17','2017-05-01','2017-05-03','2017-05-30','2017-07-01',
           '2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26','2018-01-01',
           '2018-02-16','2018-02-17','2018-02-19','2018-03-30','2018-03-31','2018-04-02',
           '2018-04-05','2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25',
           '2018-10-01','2018-10-17','2018-12-25','2018-12-26']
    dataset['holiday']=dataset['datetime'].apply(lambda x:1 if x.strftime('%Y-%m-%d') in holiday_list else 0)
    return dataset
    
def dataset_transformation_operations(train_dataset): 
    train_dataset=datetime_transfermation(train_dataset)
    train_dataset=get_day_month_year(train_dataset)
    train_dataset=get_hour(train_dataset)
    train_dataset=get_Weekday(train_dataset)
    train_dataset=get_weekday_second_edition(train_dataset)
    train_dataset=get_hour_feature(train_dataset,True)
    train_dataset=get_public_holiday(train_dataset)
    return train_dataset

In [6]:
'''
Do Same Operation to both train_data and test_data
'''
train_dataset=dataset_transformation_operations(train_dataset)
test_dataset=dataset_transformation_operations(test_dataset)

In [7]:
print(train_dataset.head())

   id           date      speed            datetime  month  day  year  hour  \
0   0  1/1/2017 0:00  43.002930 2017-01-01 00:00:00      1    1  2017     0   
1   1  1/1/2017 1:00  46.118696 2017-01-01 01:00:00      1    1  2017     1   
2   2  1/1/2017 2:00  44.294158 2017-01-01 02:00:00      1    1  2017     2   
3   3  1/1/2017 3:00  41.067468 2017-01-01 03:00:00      1    1  2017     3   
4   4  1/1/2017 4:00  46.448653 2017-01-01 04:00:00      1    1  2017     4   

   Sun  Sat  Fri  Thu  Wed  Tue  Mon  weekday hour_t  holiday  
0    1    0    0    0    0    0    0        6      a        0  
1    1    0    0    0    0    0    0        6      a        0  
2    1    0    0    0    0    0    0        6      a        0  
3    1    0    0    0    0    0    0        6      a        0  
4    1    0    0    0    0    0    0        6      b        0  


In [8]:
def set_timezone(dt):
    cst_tz = timezone('Asia/Shanghai')  
    utc_tz = timezone('Asia/Shanghai') 
    dt["datetime"]=dt["datetime"].apply(lambda x:x.replace(tzinfo=utc_tz).astimezone(cst_tz))
    return dt

def get_merge_result(dt,data_2017,data_2018):
    data_2017.drop_duplicates(subset=['datetime'],keep='first',inplace=True)
    data_2018.drop_duplicates(subset=['datetime'],keep='first',inplace=True)
    result=pd.merge(dt,data_2017,how='left',on=['datetime'])
    result=pd.merge(result,data_2018,how='left',on=['datetime'])
    return result
'''
Merge Tables include temperature information and table include other information(speed, datetime, and so on).
'''
train_dataset=set_timezone(train_dataset)
print(train_dataset.head())

   id           date      speed                  datetime  month  day  year  \
0   0  1/1/2017 0:00  43.002930 2017-01-01 00:00:00+08:00      1    1  2017   
1   1  1/1/2017 1:00  46.118696 2017-01-01 01:00:00+08:00      1    1  2017   
2   2  1/1/2017 2:00  44.294158 2017-01-01 02:00:00+08:00      1    1  2017   
3   3  1/1/2017 3:00  41.067468 2017-01-01 03:00:00+08:00      1    1  2017   
4   4  1/1/2017 4:00  46.448653 2017-01-01 04:00:00+08:00      1    1  2017   

   hour  Sun  Sat  Fri  Thu  Wed  Tue  Mon  weekday hour_t  holiday  
0     0    1    0    0    0    0    0    0        6      a        0  
1     1    1    0    0    0    0    0    0        6      a        0  
2     2    1    0    0    0    0    0    0        6      a        0  
3     3    1    0    0    0    0    0    0        6      a        0  
4     4    1    0    0    0    0    0    0        6      b        0  


In [9]:
result=get_merge_result(train_dataset,tmp_data_2017,tmp_data_2018)
print(len(result))
print(result.head())

14006
   id           date      speed                  datetime  month  day  year  \
0   0  1/1/2017 0:00  43.002930 2017-01-01 00:00:00+08:00      1    1  2017   
1   1  1/1/2017 1:00  46.118696 2017-01-01 01:00:00+08:00      1    1  2017   
2   2  1/1/2017 2:00  44.294158 2017-01-01 02:00:00+08:00      1    1  2017   
3   3  1/1/2017 3:00  41.067468 2017-01-01 03:00:00+08:00      1    1  2017   
4   4  1/1/2017 4:00  46.448653 2017-01-01 04:00:00+08:00      1    1  2017   

   hour  Sun  Sat  Fri  Thu  Wed  Tue  Mon  weekday hour_t  holiday  Temp_x  \
0     0    1    0    0    0    0    0    0        6      a        0     NaN   
1     1    1    0    0    0    0    0    0        6      a        0     NaN   
2     2    1    0    0    0    0    0    0        6      a        0     NaN   
3     3    1    0    0    0    0    0    0        6      a        0     NaN   
4     4    1    0    0    0    0    0    0        6      b        0     NaN   

   Temp_y  
0     NaN  
1     NaN  
2     Na

In [10]:
#print(len(train_dataset))
#print(result["Temp_x"])
#print(result["Temp_y"])
a=list(result["Temp_x"])
b=list(result["Temp_y"])

In [11]:
import math
c=[]
for i in range(len(a)):
    if not math.isnan(a[i]):
        c.append(a[i])
    else:
        c.append(b[i])
for i in range(8):
    c[i]=190.0
result["Temp"]=c
print(result.head())
'''
Get Temperature information for each row in train table. 
'''

   id           date      speed                  datetime  month  day  year  \
0   0  1/1/2017 0:00  43.002930 2017-01-01 00:00:00+08:00      1    1  2017   
1   1  1/1/2017 1:00  46.118696 2017-01-01 01:00:00+08:00      1    1  2017   
2   2  1/1/2017 2:00  44.294158 2017-01-01 02:00:00+08:00      1    1  2017   
3   3  1/1/2017 3:00  41.067468 2017-01-01 03:00:00+08:00      1    1  2017   
4   4  1/1/2017 4:00  46.448653 2017-01-01 04:00:00+08:00      1    1  2017   

   hour  Sun  Sat  ...  Thu  Wed  Tue  Mon  weekday  hour_t holiday  Temp_x  \
0     0    1    0  ...    0    0    0    0        6       a       0     NaN   
1     1    1    0  ...    0    0    0    0        6       a       0     NaN   
2     2    1    0  ...    0    0    0    0        6       a       0     NaN   
3     3    1    0  ...    0    0    0    0        6       a       0     NaN   
4     4    1    0  ...    0    0    0    0        6       b       0     NaN   

   Temp_y   Temp  
0     NaN  190.0  
1     NaN  1

'\nGet Temperature information for each row in train table. \n'

In [12]:
category_vars=['hour_t','month','day','holiday','Sun']
number_vars=["Temp",'year','Sat','Mon','Fri','weekday','hour']
'''
Features will be used in training stage.
'''

'\nFeatures will be used in training stage.\n'

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

n_transformation=Pipeline(steps=[('scaler', StandardScaler())])
c_transformation=Pipeline(steps=[('oneHot',OneHotEncoder())])

preprocessor=ColumnTransformer(transformers=[('num',n_transformation,number_vars),
    ('cat',c_transformation,category_vars)])

train_dataset_m=preprocessor.fit_transform(result)

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_dataset_m,train_dataset['speed'],test_size=0.1,random_state=980521)
'''
10% of data are used for testing.
'''

'\n10% of data are used for testing.\n'

In [15]:
print(X_train.shape)

(12605, 61)


In [16]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as MSE

model = XGBRegressor()

param_grid = {'nthread':[4], 
              'objective':['reg:squarederror'],
              'learning_rate':[.03,.05],
              'max_depth':[4,5,6],
              'min_child_weight':[2,3,4,5],
              'subsample':[0.8],
              'colsample_bytree':[0.8],
              'gamma':[0],
              'reg_lambda':[1],
              'reg_alpha':[0],
              'n_estimators':[1200]}
'''
After many attempts, we finally determined a smaller range for grid search to obtain the best parameters
'''
GS=GridSearchCV(estimator=model,param_grid=param_grid,cv=4,verbose=4,n_jobs=2)
GS.fit(X_train,y_train)
y_pred=GS.predict(X_test)


Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:  1.1min
[Parallel(n_jobs=2)]: Done  96 out of  96 | elapsed:  5.4min finished
D:\Anaconda\Ana\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [17]:
print("{0}".format(GS.best_params_))
print(MSE(y_pred,y_test))

{'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.03, 'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 1200, 'nthread': 4, 'objective': 'reg:squarederror', 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.8}
12.762005160984351


In [18]:
test_dataset=set_timezone(test_dataset)
test_result=get_merge_result(test_dataset,tmp_data_2017,tmp_data_2018)
a=list(test_result["Temp_x"])
b=list(test_result["Temp_y"])
import math
c=[]
for i in range(len(a)):
    if not math.isnan(a[i]):
        c.append(a[i])
    else:
        c.append(b[i])
        
test_result["Temp"]=c

'''
Do same operation to test data.
'''

test_dataset_m=preprocessor.fit_transform(test_result)
y_pred_test=GS.predict(test_dataset_m)
y_p=list(y_pred_test)
pred=pd.DataFrame({'speed':y_p},columns=['speed'])
result=pd.concat([test_data.drop('date',axis=1),pred],axis=1)
result.to_csv('E:\\5001\\KAGGLE\\Result_BOGUS_final.csv',index=False)
'''
Get and Save the final result.
'''

'\nGet and Save the final result.\n'